In [13]:
!pip install boto3

  Using cached botocore-1.17.47-py2.py3-none-any.whl (6.5 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.17.44
    Uninstalling botocore-1.17.44:
      Successfully uninstalled botocore-1.17.44
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

aiobotocore 1.1.0 requires botocore<1.17.45,>=1.17.44, but you'll have botocore 1.17.47 which is incompatible.


In [12]:
!pip install s3fs

  Using cached botocore-1.17.44-py2.py3-none-any.whl (6.5 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.17.47
    Uninstalling botocore-1.17.47:
      Successfully uninstalled botocore-1.17.47
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

boto3 1.14.47 requires botocore<1.18.0,>=1.17.47, but you'll have botocore 1.17.44 which is incompatible.


In [96]:
from scipy import io as sio
import numpy as np
import csv
import matplotlib as mp
import matplotlib.pyplot as plt
from numpy.lib import recfunctions
import datetime
import numpy.lib.recfunctions as recfn
import pandas as pd
import glob
import json
import s3fs
import io
import gzip
import csv
import datetime

In [97]:
import boto3
s3 = boto3.resource('s3')
def s3_bucket_object_keys(bucket_name= 'iiaweb-s3-io-practice-bucket'):
    bucket = s3.Bucket(bucket_name)
    key_list=[]
    for key in bucket.objects.all():
        key_list.append(key)
    return(key_list)

In [98]:
from io import BytesIO
def load_mat_file(s3_key, bucketname= 'iiaweb-s3-io-practice-bucket'):
    s3 = boto3.resource('s3')
    obj = s3.Object(bucketname, s3_key)
    inMATFile = obj.get()['Body'].read()
    raw_mat = sio.loadmat(BytesIO(inMATFile))
    return(raw_mat)

In [99]:
start = datetime.datetime.now()
csv.gzip_list = s3_bucket_object_keys('iiaweb-s3-io-practice-bucket')
end = datetime.datetime.now()
print("Time Taken to run the funtion ",str(end-start))

Time Taken to run the funtion  0:00:16.389294


In [100]:
start = datetime.datetime.now()
mat_list = s3_bucket_object_keys('iia-vault-telemetry-practice-unzipped')
end = datetime.datetime.now()
print("Time Taken to run the funtion ",str(end-start))

Time Taken to run the funtion  0:00:20.663182


In [101]:
mat_list[0]

s3.ObjectSummary(bucket_name='iia-vault-telemetry-practice-unzipped', key='Flight 652/1/652200101092009.mat')

In [131]:
import pandas as pd
out_put = pd.Series(csv.gzip_list)
# out_put.columns = 'key_name'
in_put = pd.Series(mat_list)

In [132]:
def extract_details(x):
    key= x.key
    try:
        flight, folder,filename= key.split('/')
        idx= filename.split('.')[0]
        return(idx)
    except ValueError:
        return(key)

In [133]:
in_file= in_put.apply(lambda x: extract_details(x))
in_put = pd.concat([in_file, in_put], axis=1)
in_put.columns = ['flight','S3key']

In [134]:
out_file= out_put.apply(lambda x: extract_details(x))
out_put = pd.concat([out_file, out_put], axis=1)
out_put.columns = ['flight','S3key']

In [138]:
type(out_put)

pandas.core.frame.DataFrame

In [143]:
lifecycle= in_put.merge(out_put, on = 'flight', how= 'outer', suffixes=('_mat_in', '_csv_out'))

In [144]:
lifecycle.shape

(74009, 3)

In [145]:
lifecycle.head()

,flight,S3key_mat_in,S3key_csv_out
0,652200101092009,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
1,652200101092046,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
2,652200101092046,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
3,652200101100441,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
4,652200101111519,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...


In [153]:
tailno = '652'
idx= lifecycle.flight.str[0:3]== tailno
lifecycle.loc[idx]

,flight,S3key_mat_in,S3key_csv_out
0,652200101092009,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
1,652200101092046,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
2,652200101092046,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
3,652200101100441,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
4,652200101111519,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
...,...,...,...
4426,652201003150642,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
4427,652201003151027,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
4428,652201003151332,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...
4429,652201003151610,s3.ObjectSummary(bucket_name='iia-vault-teleme...,s3.ObjectSummary(bucket_name='iiaweb-s3-io-pra...


In [154]:
def find_tail(tailno):
    idx= lifecycle.flight.str[0:3]== tailno
    return(lifecycle.loc[idx])

In [155]:
df= find_tail(tailno)